In [1]:
import pandas as pd
import numpy as np
from nsepy import get_history
import datetime
from datetime import date, timedelta, datetime
import matplotlib.pyplot as plt
import time
from dateutil.relativedelta import relativedelta
from datetime import date

In [2]:
def index_to_date(bt_data):
    new_index = []
    for index in bt_data.index:
        temp = index.to_pydatetime().date()
        new_index.append(temp)
    bt_data.index = new_index
    return bt_data

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [3]:
def hist_data(factor):
    
    if factor == "Momentum" or factor == "momentum" or factor == "m":
        bt_data = pd.read_excel(r"Nifty200DailyReturns.xlsx", sheet_name = "Nifty 200 Return-16 Year")[1:]
        bt_data.set_index("Date", inplace = True)
        bt_data = index_to_date(bt_data)
        
    if factor == "Beta" or factor == "beta" or factor == "b":
        daily_p = pd.read_excel(r"Nifty200DailyReturns.xlsx", sheet_name = "BBG-Nifty 200 Prices-16 Year").iloc[:,1:]
        
        new_header = daily_p.iloc[0] 
        daily_p = daily_p[1:]
        daily_p.columns = new_header
        daily_p.set_index("Date", inplace = True)

        weekly_p = daily_p.resample('W').last()
        weekly_r = weekly_p.pct_change()[1:]
        bt_data = weekly_r
        bt_data = index_to_date(bt_data)
        
    if factor == "BetaNifty" or factor == "betanifty" or factor == "bn":
        daily_p = pd.read_excel(r"Nifty200DailyReturns.xlsx", sheet_name = "Nifty 50 Index Prices-16 Year").iloc[:,1:]
        
        new_header = daily_p.iloc[0] 
        daily_p = daily_p[1:]
        daily_p.columns = new_header
        daily_p.set_index("Date", inplace = True)

        weekly_p = daily_p.resample('W').last()
        weekly_r = weekly_p.pct_change()[1:]
        bt_data = weekly_r
        bt_data = index_to_date(bt_data)
        
    if factor == "nifty" or factor == "Nifty":
        bt_data = pd.read_excel(r"Nifty200DailyReturns.xlsx", sheet_name = "Nifty 50 Index Return-16 Year")[1:]
        bt_data.set_index("Date", inplace = True)
        bt_data = index_to_date(bt_data)
        
    if factor == "volatility" or factor == "Volatility" or factor == "vol":
        bt_data = pd.read_excel(r"Nifty200DailyReturns.xlsx", sheet_name = "Nifty 200 Return-16 Year")[1:]
        bt_data.set_index("Date", inplace = True)
        bt_data = index_to_date(bt_data)
    return bt_data

In [4]:
all_data_momentum = hist_data("Momentum")
all_data_volatility = hist_data("Volatility")
all_data_beta = hist_data("Beta")
all_data_beta_nifty = hist_data("BetaNifty")

In [5]:
winsorize_pct = 0.02

In [235]:
portfolio_date = (datetime.today().date() - timedelta(days = 360)).replace(day=1)
portfolio_date

datetime.date(2020, 3, 1)

In [236]:
def outliers(final_df, factor):
    if factor == "Momentum" or factor == "momentum" or factor == "m":
        y = final_df['Momentum']
        outliers_downside = y.between(y.quantile(0), y.quantile(winsorize_pct))
        outliers_upside = y.between(y.quantile(1-winsorize_pct), y.quantile(1))
        low_cutoff_percentile_momentum = y.quantile(winsorize_pct)
        high_cutoff_percentile_momentum = y.quantile(1-winsorize_pct)
        outliers_downside_index = final_df[outliers_downside].index
        outliers_upside_index = final_df[outliers_upside].index
        for ind in outliers_downside_index:
            final_df.loc[ind, "Momentum"] = low_cutoff_percentile_momentum
        for ind in outliers_upside_index:
            final_df.loc[ind, "Momentum"] = high_cutoff_percentile_momentum
        
    if factor == "volatility" or factor == "Volatility" or factor == "vol":
        y = final_df["Volatility"]
        outliers_downside = y.between(y.quantile(0), y.quantile(winsorize_pct))
        outliers_upside = y.between(y.quantile(1-winsorize_pct), y.quantile(1))
        low_cutoff_percentile_volatility = y.quantile(winsorize_pct)
        high_cutoff_percentile_volatility = y.quantile(1-winsorize_pct)
        outliers_downside_index = final_df[outliers_downside].index
        outliers_upside_index = final_df[outliers_upside].index
        for ind in outliers_downside_index:
            final_df.loc[ind, "Volatility"] = low_cutoff_percentile_volatility
        for ind in outliers_upside_index:
            final_df.loc[ind, "Volatility"] = high_cutoff_percentile_volatility        

    if factor == "beta" or factor == "Beta":
        y = final_df["Beta"]
        outliers_downside = y.between(y.quantile(0), y.quantile(winsorize_pct))
        outliers_upside = y.between(y.quantile(1-winsorize_pct), y.quantile(1))
        low_cutoff_percentile_beta = y.quantile(winsorize_pct)
        high_cutoff_percentile_beta = y.quantile(1-winsorize_pct)
        outliers_downside_index = final_df[outliers_downside].index
        outliers_upside_index = final_df[outliers_upside].index
        for ind in outliers_downside_index:
            final_df.loc[ind, "Beta"] = low_cutoff_percentile_beta
        for ind in outliers_upside_index:
            final_df.loc[ind, "Beta"] = high_cutoff_percentile_beta                
            
    return final_df

In [237]:
def score(final_df):
    score = []
    for i in range(len(final_df)):
        if final_df.iloc[i]["Zscore"] >= 0:
            score.append(1 + final_df.iloc[i]["Zscore"])
        elif final_df.iloc[i]["Zscore"] < 0:
            score.append((1 - final_df.iloc[i]["Zscore"])**(-1))
        else:
            score.append(np.nan)
    return score

In [238]:
def get_relevant_returns(factor, all_data):
    if factor == "Momentum":
        rvrsl_prd = 1
        relevant_end_date = nearest(all_data.index,(portfolio_date - pd.DateOffset(months=rvrsl_prd)).to_pydatetime().date())
        relevant_start_date = nearest(all_data.index, (portfolio_date - pd.DateOffset(months=12)).to_pydatetime().date())
        relevant_returns = all_data.loc[relevant_start_date:relevant_end_date]
        
    if factor == "Volatility":
        relevant_start_date = nearest(all_data.index,(portfolio_date - pd.DateOffset(years=2)).to_pydatetime().date())
        relevant_end_date = nearest(all_data.index, (portfolio_date))
        relevant_returns = all_data.loc[(all_data.index>relevant_start_date)&(all_data.index<relevant_end_date)]
        
    if factor == "Beta":
        relevant_start_date = nearest(all_data.index,(portfolio_date - pd.DateOffset(years=1)).to_pydatetime().date())
        relevant_end_date = nearest(all_data.index, (portfolio_date))
        relevant_returns = all_data.loc[(all_data.index>relevant_start_date)&(all_data.index<relevant_end_date)]
        
    return relevant_returns

In [239]:
def factor_calculation(factor, relevant_returns, bt_data):
    if factor == "Momentum":
        df_cum_returns = pd.DataFrame(columns = relevant_returns.columns)
        for ticker in relevant_returns.columns:
            df_cum_returns[ticker] = (1 + relevant_returns[ticker]).cumprod() - 1
        cumulative_returns = df_cum_returns[-1:].T
        all_sd = relevant_returns.std().dropna()
        all_sd = all_sd[~(all_sd == 0)].dropna()
        all_sd = all_sd.to_frame()
        all_sd = all_sd.reset_index().rename(columns = {"index" : "Company"})
        all_sd = all_sd.set_index("Company")
        all_sd["Ann SD"] = all_sd*np.sqrt(252)

        final_df = pd.merge(all_sd,cumulative_returns, left_index=True, right_index=True)
        columns = ["Daily SD", "Ann SD", "Cumulative Returns (12-1)"]
        final_df.columns = columns
        final_df["Momentum"] = final_df["Cumulative Returns (12-1)"]/final_df["Ann SD"]
    
    if factor == "Volatility":
        date_yr_ago = []
        for i in range(len(bt_data.index)):
            date_yr_ago.append(nearest(bt_data.index, bt_data.index[i] - relativedelta(days=365)))
        for i in range(len(relevant_returns)):
            if (relevant_returns.index[i] - date_yr_ago[i]).days > 363:
                relevant_index = i
                break
        calc_start_date = date_yr_ago[i]
        calc_end_date = relevant_returns.index[i-1]
        relevant_data_for_calc = relevant_returns.iloc[relevant_index:]
        cum_ret_df = pd.DataFrame(columns = relevant_data_for_calc.columns, index = relevant_data_for_calc.index)
        calc_start_dates = []
        calc_end_dates = []
        for i in range(len(relevant_data_for_calc)):
            calc_start_date = nearest(relevant_returns.index, (relevant_data_for_calc.index[i] - relativedelta(days = 365)))
            calc_end_date = relevant_data_for_calc.index[i]
            calc_start_dates.append(calc_start_date)
            calc_end_dates.append(calc_end_date)
        for i in range(len(cum_ret_df)):
            curr_date = cum_ret_df.index[i]
            cum_ret_df.loc[curr_date] = (((1 + relevant_returns[calc_start_dates[i]: calc_end_dates[i]]).cumprod() - 1)[-1:].values)
        final_df = pd.DataFrame(index = cum_ret_df.columns, columns = ["Max", "Min", "Volatility"])
        for i in cum_ret_df.columns:
            final_df.loc[i]["Max"] = cum_ret_df[i].max()
            final_df.loc[i]["Min"] = cum_ret_df[i].min()
            final_df.loc[i]["Volatility"] = final_df.loc[i]["Max"] - final_df.loc[i]["Min"]
        final_df["StdDev"] = np.std(bt_data.iloc[relevant_index:])
        
    if factor == "Beta":
        print("Change function call, special function for beta calculation")
    return final_df

In [240]:
def z_score(factor, final_df):
    final_df["Zscore"] = (final_df[factor] - final_df[factor].mean())/final_df[factor].std(ddof=0)
    return final_df

In [241]:
def rank_calculation(factor, score, final_df):
    if factor == "Momentum":
        final_df.rename(columns = {"Zscore": "Momentum ZScore"}, inplace = True)
        final_df["Momentum Score"] = score
        final_df["Momentum Rank"] = final_df["Momentum Score"].rank(ascending = False)
        final_df.sort_values(by = ["Momentum Rank","Cumulative Returns (12-1)"], ascending=[True,False],inplace = True)
        
    if factor == "Volatility":
        final_df.rename(columns = {"Zscore": "Volatility ZScore"}, inplace = True)
        final_df["Volatility Score"] = score
        final_df["Vol_Rank"] = final_df["Volatility Score"].rank(ascending=True)
        final_df["Std_Rank"] = final_df["StdDev"].rank(ascending=True)
        final_df['Average']= (final_df['Vol_Rank']+final_df['Std_Rank'])/2
        final_df.sort_values(by = "Average", inplace = True)
    
    if factor == "Beta":
        final_df.rename(columns = {"Zscore": "Beta ZScore"}, inplace = True)
        final_df["Beta Score"] = score
        final_df.dropna(how='any', inplace = True)
        final_df["Beta Rank"] = final_df["Beta Score"].rank(ascending = False)
        final_df.sort_values(by = ["Beta Rank"], ascending=[False],inplace = True) ##Set false for low beta
    
    return final_df

In [242]:
def nifty_beta_calculation(relevant_returns, all_data_nifty):
    relevant_start_date = relevant_returns.index[0]
    relevant_end_date = relevant_returns.index[-1]
    relevant_nifty_data = all_data_nifty.loc[(all_data_nifty.index>=relevant_start_date)&(all_data_nifty.index<=relevant_end_date)]
    beta_df = pd.DataFrame(index = relevant_nifty_data.columns, columns = ["Beta"])
    beta_dict = {}
    for company in relevant_returns.columns:
        beta_dict[company] =  pd.merge(relevant_returns[company], relevant_nifty_data, left_index=True, right_index= True)
        beta_df.loc[company] = beta_dict[company].cov()[company]["NIFTY INDEX"]/beta_dict[company].cov()["NIFTY INDEX"]["NIFTY INDEX"]
    final_df = beta_df.reset_index().rename(columns={0:"Company"}).set_index("Company")
    final_df = final_df[1:]
    final_df["Beta"] = abs(final_df["Beta"])
    return final_df

In [243]:
def normalization_multi_factor(factor, final_df, dir):
    var = factor + " Score"
    if dir == "HTB":
        final_df[var] = (final_df[var]-final_df[var].min())/(final_df[var].max()-final_df[var].min())
    if dir == "LTB":
        final_df[var] = (final_df[var].max() - final_df[var])/(final_df[var].max()-final_df[var].min())
        
    return final_df

In [244]:
def merge_multi_factor(final_df1, final_df2, factor_1, factor_2):
    final_df = pd.concat([final_df1,final_df2],axis = 1)
    var_1 = factor_1 + " Score"
    var_2 = factor_2 + " Score"
    final_df["Final Score"] = (final_df[var_1] + final_df[var_2]) / 2
    final_df["Rank"] = final_df["Final Score"].rank(ascending = False)
    final_df.sort_values(by = "Rank", inplace = True)
    
    return final_df

In [245]:
def momentum():
    factor = "Momentum"
    all_data = all_data_momentum
    relevant_returns = get_relevant_returns(factor, all_data)
    final_df = factor_calculation(factor,relevant_returns, all_data)
    final_df = outliers(final_df, factor)
    final_df = z_score(factor, final_df)
    momentum_score = score(final_df)
    final_df = rank_calculation(factor, momentum_score, final_df)        
    
    return final_df

def volatility():
    factor = "Volatility"
    all_data = all_data_volatility
    relevant_returns = get_relevant_returns(factor, all_data)
    final_df = factor_calculation(factor,relevant_returns, all_data)
    final_df = outliers(final_df, factor)
    final_df = z_score(factor, final_df)
    momentum_score = score(final_df)
    final_df = rank_calculation(factor, momentum_score, final_df)        
    
    return final_df

def beta():
    final_daily_portfolio_returns = pd.DataFrame()
    factor = "Beta"
    all_data = all_data_beta
    all_data_nifty = all_data_beta_nifty
    relevant_returns = get_relevant_returns(factor, all_data)
    final_df = nifty_beta_calculation(relevant_returns, all_data_nifty)
    final_df = outliers(final_df, factor)
    final_df = z_score(factor, final_df)
    beta_score = score(final_df)
    final_df = rank_calculation(factor, beta_score, final_df)
    
    return final_df

def momentum_volatility():
    factor_1 = "Momentum"
    factor_2 = "Volatility"
    all_data_1 = all_data_momentum
    all_data_2 = all_data_volatility

    relevant_returns = get_relevant_returns(factor_1, all_data_1)        
    final_df = factor_calculation(factor_1,relevant_returns, all_data_1)
    final_df = outliers(final_df, factor_1)
    final_df = z_score(factor_1, final_df)
    momentum_score = score(final_df)
    final_df_1 = rank_calculation(factor_1, momentum_score, final_df)
    final_df_1 = normalization_multi_factor(factor_1, final_df_1, "HTB")               #HTB=Higher the better

    relevant_returns = get_relevant_returns(factor_2, all_data_2)  
    final_df = factor_calculation(factor_2,relevant_returns, all_data_2)
    final_df = outliers(final_df, factor_2)
    final_df = z_score(factor_2, final_df)
    volatility_score = score(final_df)
    final_df_2 = rank_calculation(factor_2, volatility_score, final_df)
    final_df_2 = normalization_multi_factor(factor_2, final_df_2, "LTB")

    final_df = merge_multi_factor(final_df_1,final_df_2, factor_1, factor_2)
    
    return final_df

def momentum_beta():
    factor_1 = "Momentum"
    factor_2 = "Beta"
    all_data_1 = all_data_momentum
    all_data_2 = all_data_beta
    all_data_nifty = all_data_beta_nifty
    
    relevant_returns = get_relevant_returns(factor_1, all_data_1)        
    final_df = factor_calculation(factor_1,relevant_returns, all_data_1)
    final_df = outliers(final_df, factor_1)
    final_df = z_score(factor_1, final_df)
    momentum_score = score(final_df)
    final_df_1 = rank_calculation(factor_1, momentum_score, final_df)
    final_df_1 = normalization_multi_factor(factor_1, final_df_1, "HTB")               #HTB=Higher the better

    relevant_returns = get_relevant_returns(factor_2, all_data_2)
    final_df = nifty_beta_calculation(relevant_returns, all_data_nifty)
    final_df = outliers(final_df, factor_2)
    final_df = z_score(factor_2, final_df)
    beta_score = score(final_df)
    final_df_2 = rank_calculation(factor_2, beta_score, final_df)
    final_df_2 = normalization_multi_factor(factor_2, final_df_2, "LTB")

    final_df = merge_multi_factor(final_df_1,final_df_2, factor_1, factor_2)
    
    return final_df

def volatility_beta():
    factor_1 = "Volatility"
    factor_2 = "Beta"
    all_data_1 = all_data_volatility
    all_data_2 = all_data_beta
    all_data_nifty = all_data_beta_nifty
    
    relevant_returns = get_relevant_returns(factor_1, all_data_1)        
    final_df = factor_calculation(factor_1,relevant_returns, all_data_1)
    final_df = outliers(final_df, factor_1)
    final_df = z_score(factor_1, final_df)
    volatility_score = score(final_df)
    final_df_1 = rank_calculation(factor_1, volatility_score, final_df)
    final_df_1 = normalization_multi_factor(factor_1, final_df_1, "LTB")               #LTB=lower the better

    relevant_returns = get_relevant_returns(factor_2, all_data_2)
    final_df = nifty_beta_calculation(relevant_returns, all_data_nifty)
    final_df = outliers(final_df, factor_2)
    final_df = z_score(factor_2, final_df)
    beta_score = score(final_df)
    final_df_2 = rank_calculation(factor_2, beta_score, final_df)
    final_df_2 = normalization_multi_factor(factor_2, final_df_2, "LTB")

    final_df = merge_multi_factor(final_df_1,final_df_2, factor_1, factor_2)
    
    return final_df

In [246]:
momentum_port = momentum()

In [247]:
volatility_port = volatility()

In [248]:
beta_port = beta()

In [249]:
momentum_volatility_port = momentum_volatility()

In [250]:
momentum_beta_port = momentum_beta()

In [251]:
volatility_beta_port = volatility_beta()

In [252]:
momentum_port.to_excel(r"momentum_port_{}.xlsx".format(portfolio_date))
volatility_port.to_excel(r"volatility_port_{}.xlsx".format(portfolio_date))
beta_port.to_excel(r"beta_port_{}.xlsx".format(portfolio_date))
momentum_volatility_port.to_excel(r"momentum_volatility_port_{}.xlsx".format(portfolio_date))
momentum_beta_port.to_excel(r"momentum_beta_port_{}.xlsx".format(portfolio_date))
volatility_beta_port.to_excel(r"volatility_beta_port_{}.xlsx".format(portfolio_date))